In [44]:
import mysql.connector
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', -1)

In [10]:
connection = mysql.connector.connect(user='root', password='root', database='classicmodels')

In [11]:
def execute_query(query):
    return pd.read_sql_query(query, connection)

### Q1: Count the number of employees whose last name or first name starts with the letter 'P'.
#### There are 5 such employees

In [23]:
query1 = '''select 'P' as begin_letter, count(*) as no_of_employees from employees where firstName like "p%" or lastName like "p%" '''
execute_query(query1)

,begin_letter,no_of_employees
0,P,5


### Q2: For how many letters of the alphabet are there more than one employee whose last name starts with that letter?

#### ans: There are 7 such letters of the alphabet

In [21]:
query2 = '''select substr(lastName,1,1) as alphabet, count(*) as no_of_employees from employees group by substr(lastName,1,1) having count(*) > 1;'''
execute_query(query2)

,alphabet,no_of_employees
0,B,4
1,F,3
2,J,2
3,K,2
4,M,2
5,P,3
6,T,2


### Q3: How many orders have not yet shipped?
#### I have considered 'orders not yet shipped' as 'orders that are yet to be shipped'. Thus, I have not considered orders that are cancelled or disputed. There are total 10 such orders

In [28]:
query3 = '''select status, count(*) as count from orders where status in ('In Process', 'On Hold') group by status'''
execute_query(query3)

,status,count
0,In Process,6
1,On Hold,4


### Q4: How many orders were shipped less than 2 days before they were required?
#### Therer are 25 such orders.

In [29]:
query4= '''select * from orders where shippedDate is not NULL and datediff(requiredDate, shippedDate) < 2'''
execute_query(query4)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10111,2003-03-25,2003-03-31,2003-03-30,Shipped,None,129
1,10121,2003-05-07,2003-05-13,2003-05-13,Shipped,None,353
2,10128,2003-06-06,2003-06-12,2003-06-11,Shipped,None,141
3,10133,2003-06-27,2003-07-04,2003-07-03,Shipped,None,141
4,10149,2003-09-12,2003-09-18,2003-09-17,Shipped,None,487
5,10157,2003-10-09,2003-10-15,2003-10-14,Shipped,None,473
6,10160,2003-10-11,2003-10-17,2003-10-17,Shipped,None,347
7,10165,2003-10-22,2003-10-31,2003-12-26,Shipped,This order was on hold because customers's cre...,148
8,10185,2003-11-14,2003-11-21,2003-11-20,Shipped,None,320
9,10189,2003-11-18,2003-11-25,2003-11-24,Shipped,They want to reevaluate their terms agreement ...,205


#### Q5: For each distinct product line, what is the total dollar value of orders placed?

In [33]:
query5 = '''select pl.productLine, sum(od.priceEach*od.quantityOrdered) as total_value from productlines pl, products pr, orderdetails od where pl.productLine = pr.productLine and od.productCode = pr.productCode group by pl.productLine;'''
execute_query(query5)

,productLine,total_value
0,Classic Cars,3853922.49
1,Motorcycles,1121426.12
2,Planes,954637.54
3,Ships,663998.34
4,Trains,188532.92
5,Trucks and Buses,1024113.57
6,Vintage Cars,1797559.63


### Q6: For the first three customers in alphabetal order by name, what is the name of every product they have ordered?

In [46]:
query6 =''' select c.customerName, 
                   group_concat(p.productName) as products,
                   count(p.productName) as no_of_products
            from products p, orders o, orderdetails od, customers c 
            where c.customerNumber = o.customerNumber and 
            o.orderNumber = od.orderNumber and od.productCode = p.productCode 
            group by c.customerName order by c.customerName limit 3'''
execute_query(query6)

,customerName,products,no_of_products
0,Alpha Cognac,"1993 Mazda RX-7,1995 Honda Civic,1992 Ferrari 360 Spider red,1972 Alfa Romeo GTA,1980s Black Hawk Helicopter,1999 Yamaha Speed Boat,1941 Chevrolet Special Deluxe Cabriolet,1900s Vintage Bi-Plane,1937 Horch 930V Limousine,1940 Ford Delivery Sedan,Corsair F4U ( Bird Cage),HMS Bounty,America West Airlines B757-200,American Airlines: MD-11S,Boeing X-32A JSF,The Mayflower,The USS Constitution Ship,The Titanic,The Queen Mary,Pont Yacht",20
1,Amica Models & Co.,"1952 Alpine Renault 1300,1940 Ford Pickup Truck,1937 Lincoln Berline,1936 Mercedes-Benz 500K Special Roadster,1917 Grand Touring Sedan,1911 Ford Town Car,1932 Model A Ford J-Coupe,1928 Mercedes-Benz SSK,1913 Ford Model T Speedster,1934 Ford V8 Coupe,18th Century Vintage Horse Carriage,1917 Maxwell Touring Car,1932 Alfa Romeo 8C2300 Spider Sport,1939 Chevrolet Deluxe Coupe,1938 Cadillac V-16 Presidential Limousine,1936 Mercedes Benz 500k Roadster,1936 Chrysler Airflow,2001 Ferrari Enzo,1969 Corvair Monza,1969 Ford Falcon,1903 Ford Model A,Collectable Wooden Train,1904 Buick Runabout,18th century schooner,The Mayflower,Pont Yacht",26
2,"Anna's Decorations, Ltd","1993 Mazda RX-7,1965 Aston Martin DB5,1995 Honda Civic,1999 Indy 500 Monte Carlo SS,1992 Ferrari 360 Spider red,1948 Porsche Type 356 Roadster,1932 Alfa Romeo 8C2300 Spider Sport,1957 Ford Thunderbird,1970 Chevy Chevelle SS 454,1966 Shelby Cobra 427 S/C,1949 Jaguar XK 120,1952 Citroen-15CV,1969 Chevrolet Camaro Z28,2002 Chevy Corvette,1968 Ford Mustang,2002 Suzuki XREO,1970 Plymouth Hemi Cuda,1969 Dodge Super Bee,1976 Ford Gran Torino,1957 Vespa GS150,1957 Corvette Convertible,1982 Ducati 900 Monster,1971 Alpine Renault 1600s,1961 Chevrolet Impala,1982 Ducati 996 R,1974 Ducati 350 Mk3 Desmo,2002 Yamaha YZR M1,1962 LanciaA Delta 16V,1958 Setra Bus,1940 Ford Pickup Truck,1964 Mercedes Tour Bus,1926 Ford Fire Engine,1992 Ferrari 360 Spider red,1940s Ford truck,1939 Cadillac Limousine,1996 Peterbilt 379 Stake Bed with Outrigger,1952 Alpine Renault 1300,1962 LanciaA Delta 16V,1958 Setra Bus,1940 Ford Pickup Truck,1937 Lincoln Berline,1936 Mercedes-Benz 500K Special Roadster,1926 Ford Fire Engine,1913 Ford Model T",46


### Using another query, we can find out the total distinct products ordered by these customers

In [47]:
query6_b = '''select productName from products where productCode in (select productCode from orderdetails where orderNumber in(select orderNumber from orders where customerNumber in(select customerNumber from (select customerNumber from customers order by customerName limit 3) q)))
'''
execute_query(query6_b)

,productName
0,1952 Alpine Renault 1300
1,1972 Alfa Romeo GTA
2,2001 Ferrari Enzo
3,1969 Corvair Monza
4,1969 Ford Falcon
5,1940 Ford Pickup Truck
6,1993 Mazda RX-7
7,1937 Lincoln Berline
8,1936 Mercedes-Benz 500K Special Roadster
9,1980s Black Hawk Helicopter


### Python program to output table stats

In [ ]:
import mysql.connector
import sys

if len(sys.argv) < 2:
    print "database name missing"
    exit(0)

dbname = str(sys.argv[1])

cnx = mysql.connector.connect(user='root', password='root',host='127.0.0.1',database=dbname)
cursor = cnx.cursor()
base_query  = 'show tables'
count_query = ("SELECT TABLE_NAME, TABLE_ROWS FROM `information_schema`.`tables` WHERE `table_schema`=%s")
cursor.execute(count_query, (dbname,))
row_format = "{:>15}"*2
for name, count in cursor:
    print row_format.format(*(name,count))
cnx.close()